In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('../source')

import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar

import readers.npsnow as npsnow
import trajectory
from merge_npsnow_data import get_station_list, merge_one_station
from constants import DATADIR

## Merge data for all stations beyond NP-4 and excluding NP-14

1. Load data for one station
2. Drop rows with missing wind speed and air temperature values
3. Calculate monthly data
4. Calculate annual data

In [3]:
def monthly_dataframe(df):
    dfMon = pd.DataFrame({
        'ND': df.PRECIP.resample('MS').count(),
        'Tmn': df.TAIR.resample('MS', label='left').mean(),
        'Ug': df.Ug.resample('MS', label='left').mean(),
        'DP': df.PRECIP[df.PRECIP > 0.].resample('MS').count(),
        'Dtc': df.PRECIP[df.PRECIP == 0.].resample('MS').count(),
        'Pg': df.PRECIP[df.PRECIP > 0].resample('MS').sum(),
        'Ptc': df.Ptrace.resample('MS').sum(),
        'Pwind': df.Pwind.resample('MS').sum(),
        'Pcorr': df.Pcorr.resample('MS').sum(),
        'Psnow': df.Psnow.resample('MS').sum(),
        })
    return dfMon

def annual_dataframe(df):
    dfAnn = pd.DataFrame({
        'ND': df.ND.resample('AS').sum(min_count=12),
        'Tmn': df.Tmn.resample('AS').mean(),
        'Ug': df.Ug.resample('AS').mean(),
        'DP': df.DP.resample('AS').sum(min_count=12),
        'Dtc': df.Dtc.resample('AS').sum(min_count=12),
        'Pg': df.Pg.resample('AS').sum(min_count=12),
        'Ptc': df.Ptc.resample('AS').sum(min_count=12),
        'Pwind': df.Ptc.resample('AS').sum(min_count=12),
        'Pcorr': df.Pcorr.resample('AS').sum(min_count=12),
        'Psnow': df.Psnow.resample('AS').sum(min_count=12),
        })
    return dfAnn

def process_station(sid):
    df = merge_one_station(sid, set_noprecip=False)
    df = df.dropna(axis=0, subset=['WSPD', 'TAIR', 'PRECIP', 'PTYPE'])
    df['Ptrace'] = np.where((df['PRECIP'] == 0.) & (df['PTYPE'] > 0.), 0.1, 0.)  # Set trace precipitation were PRECIP == 0
    df['PRECIP'] = df['PRECIP'].where(df['PRECIP'] > 0., 0.)
    df['PTYPE'] = df['PTYPE'].where(df['PTYPE'] > 0., 0.)

    dfMon = monthly_dataframe(df)
    dfMon['StationID'] = int(sid)
    dfAnn = annual_dataframe(dfMon)
    dfAnn['StationID'] = int(sid)
    dfAnn = dfAnn.dropna(axis=0)
    
    return dfAnn.reset_index()

# Add dropna to proc
# Add station number
# Reset index 

def plot_trajectory(lon, lat, lon2, lat2):
    
    map_proj = ccrs.NorthPolarStereo()

    fig = plt.figure(figsize=(10,10))
    ax = plt.subplot(projection=map_proj)
    ax.set_extent([-180., 180., 72., 90.], ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)

    pts = map_proj.transform_points(ccrs.PlateCarree(), lon, lat)
    xm = pts[:,0]
    ym = pts[:,1]

    pts = map_proj.transform_points(ccrs.PlateCarree(), lon2, lat2)
    xu = pts[:,0]
    yu = pts[:,1]

    ax.plot(xm, ym, label='Raw')
    ax.plot(xu, yu, label='Daily')

    ax.legend()

## Merge met and precip data, replace coordinates with updated coords and write to file

In [17]:
def combine_dataframe(station):
    '''
    Merges met and precip data, and replaces coordinates with updated coordinates
    
    Coordinates are interpolated if they are missing
    
    station - station id
    
    Returns: pandas dataframe
    '''
    df = merge_one_station(str(station), set_noprecip=False)  # Merge met and precip data
    df.index = df.index.shift(12, freq='H')  # Assign daily met to 12:00:00h

    df_pos = npsnow.read_position(os.path.join(DATADIR, 'updated_position', f'position.{station:02d}'))
    df_pos = df_pos.sort_index()
    # Need to revisit updated_coordinates and remove duplicates, but deal with it here for now
    df_pos = df_pos.drop_duplicates(keep='first')
    df_pos = df_pos[~df_pos.index.duplicated(keep=False)]  # Handles case where index duplicated but values are different

    waypoints = trajectory.to_waypoints(df_pos)
    np_drift = trajectory.Trajectory(waypoints)
    df_np_drift = np_drift.interpolate_by_date(df.index).to_dataframe()
    
    df = df.join(df_np_drift, rsuffix='_new')
    df = df.drop(['Latitude', 'Longitude'], axis=1).rename({'Longitude_new': 'Longitude', 'Latitude_new': 'Latitude'}, axis=1)
    
    return df


In [19]:
for station in [22,24,25,26,28,29,30,31]:
    print(f'Processing station {station:02d}')
    df = combine_dataframe(station)
    df.to_csv(os.path.join('/home/apbarret/Data/NPSNOW/my_combined_met',f'npmet_{station:02d}_combined.csv'))
    
#plot_trajectory(df.Longitude.values, df.Latitude.values, df.Longitude_new.values, df.Latitude_new.values)

Processing station 22
Processing station 24
Processing station 25
Processing station 26
Processing station 28
Processing station 29
Processing station 30
Processing station 31


In [7]:
df.head()

,Station_ID,Latitude,Longitude,TAIR,RH,SLP,WDIR,WSPD,TOTCLD,LOWCLD,TSURF,TMIN,TMAX,PRECIP,PTYPE,SDEPTH,Ug,Longitude_new,Latitude_new
1973-10-01 12:00:00,22.0,NaN,NaN,-17.90,83.25,1007.975,125.0,9.00,5.00,0.0,-18.25,-20.7,-15.3,-9.9,-9,18.0,6.780214,-173.825672,75.826067
1973-10-02 12:00:00,22.0,NaN,NaN,-15.35,85.25,1017.100,355.0,4.25,5.00,5.0,-16.25,-20.5,-8.1,0.2,1,19.0,3.198378,-174.067214,75.912222
1973-10-03 12:00:00,22.0,NaN,NaN,-10.45,92.25,1020.675,147.5,3.25,10.00,5.0,-10.75,-11.2,-9.7,0.2,1,19.0,2.445818,-174.311663,75.998133
1973-10-04 12:00:00,22.0,76.1055,-174.61950,-15.95,86.00,1018.275,90.0,5.50,0.00,0.0,-17.00,-18.7,-12.1,0.2,1,19.0,4.139077,-174.558677,76.083860
1973-10-05 12:00:00,22.0,76.1700,-174.80425,-13.40,88.75,1019.625,110.0,6.75,7.75,7.5,-13.50,-17.6,-10.0,0.5,1,19.0,5.079777,-174.804419,76.170045


## Identify days with trace precipitation
Yang sets daily trace precipitation to 0.1 mm

In [ ]:
df['Ptrace'] = np.where((df['PRECIP'] == 0.) & (df['PTYPE'] > 0.), 0.1, 0.)  # Set trace precipitation were PRECIP == 0
df['PRECIP'] = df['PRECIP'].where((df['PRECIP'].isna()) | (df['PRECIP'] > 0.), 0.)  # Set -9.9 to zero (no precip) but leave NaN
df['PTYPE'] = df['PTYPE'].where((df['PTYPE'].isna()) | (df['PTYPE'] > 0.), 0.)  # ditto
df['Psnow'] = df['PRECIP'].where(df['PTYPE'] == 1, 0.)

## Calculate wind correction

### Catch ratios from Yang et al (1995)
Snow
$$R = 103.10 - 8.67 W_s + 0.30 T_{max}$$
Snow and Rain
$$R = 98.56 - 6.19 W_s + 0.90 T_{max}$$
Rain and Snow
$$R = 98.13 - 3.17 W_s + 0.60 T_{min}$$
Rain
$$R = 99.99 - 4.77 W_s^{0.56}$$

In [ ]:
def cr_snow(x):
    """Catch Ratio for snow"""
    return 103.11 - 8.67*x.Ug + 0.3*x.TMAX

def cr_mixed(x):
    """Catch ratio for mixed precipitation"""
    return 96.99 -4.46*x.Ug + 0.88*x.TMAX + 0.22*x.TMIN

def cr_rain(x):
    """Catch ratio for rain"""
    return 99.99 - 4.77*(x.Ug**0.56)

catch_ratio = {
    1: cr_snow,
    2: cr_mixed,
    3: cr_rain,
    0: 0.
}

def wind_correction(x):
    if x[['PRECIP', 'PTYPE', 'Ug', 'TMAX', 'TMIN']].isna().any():
        return np.nan
    if x.PRECIP == 0.:
        return 0.
    #if x.Ug > 6.:
    #    return 0.  # Yang does not apply correction for wind speeds above 6 m/s
    cr_function = catch_ratio.get(x['PTYPE'], None)
    try:
        cr = cr_function(x)
    except TypeError:
        print (f'Unexpected PTYPE {x.PTYPE}')
    k = 100./cr
    return x.PRECIP * (k - 1.)
    

## Test wind correction function

df_test = pd.DataFrame({'PRECIP': [np.nan, 0.0, 0.2, 3.0, 1.6, 2.0],
                        'PTYPE': [np.nan, 0, 1, 2, 3, -9],
                        'Ug': [4.5, 2.0, np.nan, 3.0, 4.0, 7.0],
                        'TMAX': [-15., -10., -1., 1., -20., -23.],
                        'TMIN': [-20., -18., -9., -5., -29., -30.]})
df_test.apply(wind_correction, axis=1)

Ug, TMAX, TMIN = 7.0, -23., -30.
cr = 103.11 - 8.67*Ug + 0.3*TMAX
k = 100./cr
print (cr, k, k-1)

## Apply to merged DataFrame

In [ ]:
df['Pwind'] = df.apply(wind_correction, axis=1)
df['Pcorr'] = df['PRECIP'] + df['Ptrace'] + df['Pwind']

In [ ]:
df.head()

## Calculate monthly data

In [ ]:
dfMon = monthly_dataframe(df)
dfMon['DaysInMonth'] = [calendar.monthrange(time.year, time.month)[1] for time in dfMon.index]
dfMon = dfMon[dfMon.ND == dfMon.DaysInMonth].drop('DaysInMonth', axis=1)
dfMon['Fsnow'] = dfMon['Psnow'] / dfMon['Pg']
dfMon

In [ ]:
dfAnn = annual_dataframe(dfMon)
dfAnn['Fsnow'] = dfAnn['Psnow'] / dfAnn['Pg']
dfAnn.dropna()

## Compare with Yang

In [ ]:
yang_diri = '/home/apbarret/Data/NPSNOW/yang_precip'
yangMon = npsnow.read_yang_updated(os.path.join(yang_diri, f'yang_np_precip_updated_coords_{sid}.csv'))
yangMon.index = yangMon.Date
#yangMon.index = yangMon.index.shift(12, freq='H')
yangMon = yangMon.drop('Date', axis=1)
yangMon

In [ ]:
x = yangMon.join(dfMon, rsuffix='_new')
x

In [ ]:
x.columns

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15,15))

for iax, xname, yname in zip(ax.flatten(), ['Pg', 'traceC', 'windC', 'Pc'], ['Pg_new', 'Ptc', 'Pwind', 'Pcorr']):
    x.plot(kind='scatter', x=xname, y=yname, ax=iax)
    xmax = x[[xname, yname]].max().max()
    iax.set_xlim(0, xmax)
    iax.set_ylim(0.,xmax)
    iax.set_aspect('equal')
    iax.plot([0.,xmax], [0.,xmax], c='0.5')